# The Art of Prompt Design: Prompt Boundaries and Token Healing

This (written jointly with <a href="https://medium.com/@marcotcr">Marco Tulio Ribeiro</a>) is part 2 of a series on <b>the art of prompt design</b> (part 1 <a href="https://medium.com/towards-data-science/the-art-of-prompt-design-use-clear-syntax-4fc846c1ebd5">here</a>), where we talk about controlling large language models (LLMs) with <a href="https://github.com/microsoft/guidance">`guidance`</a>.

In this post, we'll discuss how the greedy tokenization methods used by language models can introduce unintended token splits into your prompts, leading to puzzling generations.

Language models are not trained on raw text, but rather on tokens, which are chunks of text that often occur together, similar to words. This impacts how language models 'see' text, including prompts (since prompts are just sets of tokens). GPT-style models utilize tokenization methods like [Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding) (BPE), which map all input bytes to token ids in a greedy manner. This is fine for training, but it can lead to subtle issues during inference, as shown in the example below.

<!-- TODO
Standard greedy token mapping works well during training, but it can lead to subtle issues during prompting and inference. These issues arise because the greedy token boundaries often don't line up with the end of the prompt, especially when considering the generated tokens that will come next. While the end of a prompt will always align with a token boundary in practice, as the prompt is tokenized before being extended by the model, there may be instances where the first characters of the completion are part of a longer token that would span the prompt boundary. In such cases, the longer token cannot be used even though the model would expect it based on the training data.

The inability to use tokens that span prompt boundaries can lead to subtle yet important biases in the model's output. -->


## An example of a prompt boundary problem
Consider the following example, where we are trying to generate an HTTP URL string:

In [1]:
import guidance

# we use StableLM as an example, but these issues impact all models to varying degrees
guidance.llm = guidance.llms.Transformers("stabilityai/stablelm-base-alpha-3b", device=0)

# we turn token healing (discussed later) off so that guidance acts like a normal prompting library
program = guidance('''The link is <a href="http:{{gen max_tokens=10 token_healing=False}}''')
program()

Stop program The link is <a href="http: //www.google.com/search?q

Note that the output generated by the LLM does not complete the url with the obvious next characters (two forward slashes). It instead creates an invalid URL string with a space in the middle. This is surprising, because the `//` completion is extremely obvious after `http:`. To understand why this happens, let's change our prompt boundary so that our prompt does not include the colon character:

In [5]:
guidance('''The link is <a href="http{{gen max_tokens=10 token_healing=False}}''')()

Stop program The link is <a href="http ://www.youtube.com/v/s

Now the language model generates a valid url string like we expect. To understand why the  `:` matters, we need to look at the tokenized representation of the prompts. Below is the tokenization of the prompt that ends in a colon (the prompt without the colon has the same tokenization, except for the last token):

In [3]:
def print_tokens(tokens):
    print("len = " + str(len(tokens)))
    for i in tokens:
        print(str(i) + "\t`" + guidance.llm.decode([i]) + "`")

print_tokens(guidance.llm.encode('The link is <a href="http:'))

len = 9
510	`The`
3048	` link`
310	` is`
654	` <`
66	`a`
3860	` href`
568	`="`
2413	`http`
27	`:`


Now note what the tokenization of a valid URL looks like, paying careful attention to token `1358`, right after `http`:

In [17]:
print_tokens(guidance.llm.encode('The link is <a href="http://www.google.com/search?q'))

len = 18
510	`The`
3048	` link`
310	` is`
654	` <`
66	`a`
3860	` href`
568	`="`
2413	`http`
1358	`://`
2700	`www`
15	`.`
9906	`google`
15	`.`
681	`com`
16	`/`
8716	`search`
32	`?`
82	`q`


This particular LLM uses a greedy tokenization method, always preferring the longest possible token, i.e. `://` will always be preferred over `:` in full text (e.g. in training).

While URLs in training are encoded with token 1358 (`://`), our prompt makes the LLM see token `27` (`:`) instead, which throws off completion by artificially splitting `://`.
In fact, the model can be pretty sure that seeing token `27` (`:`) means what comes next is very unlikely to be anything that could have been encoded together with the colon using a "longer token" like `://`, since in the model's training data those characters would have been encoded together with the colon (an exception to this that we will discuss later is <a href="https://arxiv.org/abs/1804.10959">subword regularization</a> during training). The fact that seeing a token means both seeing the embedding of that token **and also** that whatever comes next wasn't compressed by the greedy tokenizer is easy to forget, but it is important in prompt boundaries.

Let's search over the string representation of all the tokens in the model's vocabulary, to see which ones start with a colon:

In [18]:
print_tokens(guidance.llm.prefix_matches(":"))

len = 34
27	`:`
21610	`:/`
1358	`://`
1450	`::`
41210	`::::`
5136	`:"`
46064	`:")`
18031	`:"){`
49777	`:",`
27506	`:*`
6098	`:**`
48471	`:**]{}`
8048	`:\`
10477	`:(`
13522	`:=`
25942	`:=\`
18459	`:#`
19282	`:</`
21382	`:[`
22314	`:-`
42841	`:--`
22426	`:'`
23338	`:_`
25731	`:@"`
27976	`:%`
30337	`:``
34417	`:]`
35490	`:$`
47279	`:$$\`
37731	`:)`
41924	`:{`
46186	`:{\`
43118	`:.`
44662	`:&`


Note that there are **34** different tokens starting with a colon, and thus ending a prompt with a colon means the model will likely not generate completions with any of these 34 token strings. *This subtle and powerful bias can have all kinds of unintended consequences.* And this applies to **any** string that could be potentially extended to make a longer single token (not just `:`).  Even our "fixed" prompt ending with "http" has a built in bias as well, as it communicates to the model that what comes after "http" is likely not "s" (otherwise "http" would not have been encoded as a separate token):

In [19]:
print_tokens(guidance.llm.prefix_matches("http"))

len = 2
2413	`http`
3614	`https`


Lest you think this is an arcane problem that only touches URLs, remember that most tokenizers treat tokens differently depending on whether they start with a space, punctuation, quotes, etc, and thus **ending a prompt with any of these can lead to wrong token boundaries**, and break things:

In [20]:
# Accidentally adding a space, will lead to weird generation
program = guidance('''I read a book about {{gen max_tokens=5 token_healing=False temperature=0}}''')
program()

Stop program I read a book about ~~the~~ the history

In [21]:
# No space, works as expected
program = guidance('''I read a book about{{gen max_tokens=5 token_healing=False temperature=0}}''')
program()

Stop program I read a book about the history of the New

Another example of this is the "[" character. Consider the following prompt and completion:

In [22]:
guidance('''An example ["like this"] and another example [{{gen max_tokens=10 token_healing=False}}''', caching=False)()

Stop program An example ["like this"] and another example [ like this] are shown in FIG. 1.

Why is the second string not quoted? Because by ending our prompt with the ' [' token, we are telling the model that it should not generate completions that match the following 27 longer tokens (one of which adds the quote character, `15640`):

In [23]:
print_tokens(guidance.llm.prefix_matches(" ["))

len = 27
544	` [`
1008	` [@`
3921	` [*`
4299	` [**`
23734	` [****,`
8168	` []`
24345	` [],`
26991	` [];`
27501	` []{`
8605	` [[`
44965	` [[*`
14412	` ['`
15640	` ["`
16731	` [$`
20629	` [$\`
21810	` [(`
49824	` [(\[`
21938	` […]`
24430	` [\`
27075	` [^`
28591	` [-`
31789	` [...]`
33440	` [{`
42989	` [_`
43521	` [<`
44308	` [``
49193	` [#`


Token boundary bias happens everywhere. *Over 70% of the 10k most common tokens for the StableLM model used above are prefixes of longer possible tokens, and so cause token boundary bias when they are the last token in a prompt.* Keeping track of all these possible extension biases during prompt design is impractical so most people just ignore them.

In [ ]:
# count the number of tokens that have longer extensions
count = 0
for i in range(10000):
    m = guidance.llm.prefix_matches(guidance.llm.decode([i]))
    if len(m) > 1:
        count += 1
print(str(100*count/10000)+"%")

70.23%


## Fixing unintended bias with "token healing"

What can we do to avoid these unintended biases? One option is to always end our prompts with tokens that cannot be extended into longer tokens (for example a role tag for chat-based models), but this is a severe limitation.  

Instead, `guidance` has a feature called "token healing", which automatically backs up the generation process by one token before the end of the prompt, then constrains the first token generated to have a prefix that matches the last token in the prompt. In our URL example, this would mean removing the `:`, and forcing generation of the first token to have a `:` prefix.   
Token healing allows users to express prompts however they wish, without worrying about token boundaries.

For example, let's re-run some of the URL examples above with token healing turned on (it's on by default for Transformer models, so we remove `token_healing=False`):

In [24]:
# With token healing we generate valid URLs, even when the prompt ends with a colon:
guidance('''The link is <a href="http:{{gen max_tokens=10}}''')()

Stop program The link is <a href="http: //www.youtube.com/v/s_

In [25]:
# With token healing, we'll sometimes generate https URLs, even when the prompt ends with "http":
program = guidance('''The link is <a href="http{{gen 'completions' max_tokens=10 n=10 temperature=1}}''')
program()["completions"]

Stop program The link is <a href="http <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_7a17c466d3884f4b9c45e4e85fdde7aa(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("7a17c466d3884f4b9c45e4e85fdde7aa_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("7a17c466d3884f4b9c45e4e85fdde7aa_" + (i+1)); if (!next_el) { next_el = document.getElementById("7a17c466d3884f4b9c45e4e85fdde7aa_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 10) + 1) + "/" + 10;}cycle_7a17c466d3884f4b9c45e4e85fdde7aa(this);'>1/10 ://book.info/bookmark.php? s://www.google.com/search?q ://www.sourceforge.net/mailarchive s://twitter.com/share"?&gt; ://www.wqam.com/click ://gizmos.com/tracks">G s://github.com/hongbin-v ://addressbook.nl"><img src="http ://www
whatchallup.com/ ://www.youtube.com/v/w

['://book.info/bookmark.php?',
 's://www.google.com/search?q',
 '://www.sourceforge.net/mailarchive',
 's://twitter.com/share"?&gt;',
 '://www.wqam.com/click',
 '://gizmos.com/tracks">G',
 's://github.com/hongbin-v',
 '://addressbook.nl"><img src="http',
 '://www\nwhatchallup.com/',
 '://www.youtube.com/v/w']

Similarly, we don't have to worry about extra spaces:

In [28]:

# Accidentally adding a space will not impact generation
program = guidance('''I read a book about {{gen max_tokens=5 temperature=0}}''')
program()

Stop program I read a book about the history of the New Orleans

In [30]:
# This will generate the same text as above 
program = guidance('''I read a book about{{gen max_tokens=6 temperature=0}}''')
program()

Stop program I read a book about the history of the New Orleans

Similarly, we now get quoted strings even when the prompt ends with a " [" token:

In [31]:
guidance('''An example ["like this"] and another example [{{gen max_tokens=10}}''', caching=False)()

Stop program An example ["like this"] and another example [ "like that"] are used to illustrate the invention.

## What about subword regularization?

If you are familiar with how language models are trained, you may be wondering how <a href="https://arxiv.org/abs/1804.10959">subword regularization</a> fits into all this. Subword regularization is a technique where during training sub-optimial tokenizations are randomly introduced to increase the model's robustness to token boundary issues. This means that the model does not always see the best greedy tokenization. Subword regularization is great at helping the model be more robust to token boundaries, but it does not remove the bias that the model has towards the standard greedy tokenization. This means that while depending on the amount of subword regularization during training models may exhibit more or less token boundaries bias, all models still have this bias. And as shown above it can still have a powerful and unexpected impact on the model output.

## Conclusion

When you write prompts, remember that greedy tokenization can have a significant impact on how language models interpret your prompts, particularly when the prompt ends with a token that could be extended into a longer token. This easy-to-miss source of bias can impact your results in surprising and unintended ways.

To address to this, either end your prompt with a non-extendable token, or use something like `guidance`'s "token healing" feature so you can to express your prompts however you wish, without worrying about token boundary artifacts. 

## Appendix: Did we just get unlucky with the link example?

No, and random sampling can verify that:

In [4]:
# with the colon we almost always get an invalid link
program = guidance('''The link is <a href="http:{{gen 'completions' max_tokens=10 token_healing=False temperature=1.0 n=5}}''')
program()["completions"]

Stop program The link is <a href="http: <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_020a1423a2054a428ae6c1d29d47d9f2(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("020a1423a2054a428ae6c1d29d47d9f2_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("020a1423a2054a428ae6c1d29d47d9f2_" + (i+1)); if (!next_el) { next_el = document.getElementById("020a1423a2054a428ae6c1d29d47d9f2_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_020a1423a2054a428ae6c1d29d47d9f2(this);'>1/5 
- Bookmark for Bookmark Settings </a \\www.substance.gov.uk\\ //www.facebook.com/pages/P //www.parentrepublic.org/releases //www.example.com/en/reg

['\n- Bookmark for Bookmark Settings </a',
 '\\\\www.substance.gov.uk\\\\',
 ' //www.facebook.com/pages/P',
 ' //www.parentrepublic.org/releases',
 ' //www.example.com/en/reg']

In [3]:
# without the colon we always get a valid link
program = guidance('''The link is <a href="http{{gen 'completions' max_tokens=10 token_healing=False temperature=1.0 n=5}}''')
program()["completions"]

Stop program The link is <a href="http <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_12fdfb8f14bd4d54be33f206ed2abce3(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("12fdfb8f14bd4d54be33f206ed2abce3_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("12fdfb8f14bd4d54be33f206ed2abce3_" + (i+1)); if (!next_el) { next_el = document.getElementById("12fdfb8f14bd4d54be33f206ed2abce3_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_12fdfb8f14bd4d54be33f206ed2abce3(this);'>1/5 ://www.cpmindia.com" ://www.gizmag.com/author ://www.instagram.com/oops ://www.yellow-bellied-knight ://www.gizmag.com/p

['://www.cpmindia.com"',
 '://www.gizmag.com/author',
 '://www.instagram.com/oops',
 '://www.yellow-bellied-knight',
 '://www.gizmag.com/p']